# 🏥 SimuPaciente UMH - Demo en Google Colab

**Universidad Miguel Hernández de Elche**

Sistema de simulación de pacientes virtuales con voz para práctica de entrevistas clínicas.

---

## 📋 Instrucciones

1. **Ejecuta las celdas en orden** (Shift + Enter)
2. **Proporciona tu API key de OpenAI** cuando se te solicite
3. **Accede a la URL de ngrok** que se generará al final
4. **¡Comienza a practicar!**

⚠️ **IMPORTANTE:** Este notebook usa recursos de Colab gratuitos. La sesión se cerrará automáticamente después de ~12 horas de inactividad.

---

## 🔧 Paso 1: Instalación de Dependencias

Instalamos Python, Node.js y todas las dependencias necesarias.

In [ ]:
%%bash
echo "📦 Instalando dependencias del sistema..."

# Actualizar sistema
apt-get update -qq

# Instalar Node.js 18.x
curl -fsSL https://deb.nodesource.com/setup_18.x | bash -
apt-get install -y nodejs

# Verificar versiones
echo "✅ Node.js version: $(node --version)"
echo "✅ npm version: $(npm --version)"
echo "✅ Python version: $(python3 --version)"

## 📥 Paso 2: Clonar Repositorio

Descargamos el código desde GitHub.

In [ ]:
import subprocess
import os

print("📥 Clonando repositorio desde GitHub...")

# Ir a /content primero
os.chdir('/content')

# Limpiar si existe
if os.path.exists('/content/ECOE'):
    subprocess.run(['rm', '-rf', 'ECOE'], check=True)

# Clonar repo
result = subprocess.run(
    ['git', 'clone', 'https://github.com/marcosbenghezala/ECOE.git', 'ECOE'],
    capture_output=True,
    text=True
)

if result.returncode != 0:
    print("❌ Error clonando repositorio:")
    print(result.stderr)
    raise RuntimeError("Failed to clone repository")

print("✅ Repositorio clonado exitosamente")

# Verificar que existe
if os.path.exists('/content/ECOE/simulador'):
    print("✅ Directorio /content/ECOE/simulador verificado")
    # Listar contenido
    result = subprocess.run(['ls', '-la', '/content/ECOE/'], capture_output=True, text=True)
    print(result.stdout)
else:
    print("❌ ERROR: El directorio no existe después de clonar")
    raise RuntimeError("Clone verification failed")

## 🐍 Paso 3: Instalar Dependencias Python

Instalamos las librerías de Python necesarias para el backend.

In [ ]:
%%bash
set -e  # Fallar si cualquier comando falla

# Verificar que el directorio existe
if [ ! -d "/content/ECOE/simulador" ]; then
    echo "❌ ERROR: El directorio /content/ECOE/simulador no existe"
    echo "⚠️  Asegúrate de haber ejecutado la celda de clonado (Paso 2) primero"
    exit 1
fi

cd /content/ECOE/simulador

echo "🐍 Instalando dependencias Python..."
pip install -q -r requirements.txt

echo "✅ Dependencias Python instaladas"

## ⚛️ Paso 4: Build del Frontend

Compilamos el frontend React.

In [ ]:
%%bash
set -e  # Fallar si cualquier comando falla

# Verificar que el directorio existe
if [ ! -d "/content/ECOE/simulador/frontend" ]; then
    echo "❌ ERROR: El directorio /content/ECOE/simulador/frontend no existe"
    echo "⚠️  Asegúrate de haber ejecutado la celda de clonado (Paso 2) primero"
    exit 1
fi

cd /content/ECOE/simulador/frontend

echo "⚛️ Instalando dependencias de Node.js..."
npm install --silent

echo "🔨 Building frontend..."
npm run build

echo "✅ Frontend compilado exitosamente"
ls -la /content/ECOE/simulador/frontend/dist/

## 🔑 Paso 5: Configuración del Proxy Server

**AUTOMÁTICO** - Los estudiantes NO necesitan hacer nada aquí.

La API key de OpenAI está configurada en el servidor proxy (oculta y segura).

📖 **Instrucciones para el profesor:** Ver `DEPLOYMENT_GUIDE.md` para desplegar el proxy.

In [ ]:
import os

# ============================================
# ✅ CONFIGURACIÓN AUTOMÁTICA VÍA PROXY SERVER
# ============================================
#
# Tu API key de OpenAI está en el servidor proxy
# Los estudiantes NO necesitan configurar nada
#
# ============================================

# URL del proxy server (desplegado en Render.com)
PROXY_URL = "https://simu-paciente-umh-proxy.onrender.com"

# Configurar para usar el proxy
os.environ['PROXY_URL'] = PROXY_URL

print("="*70)
print("✅ CONFIGURACIÓN COMPLETADA AUTOMÁTICAMENTE")
print("="*70)
print(f"\n🔒 Proxy server: {PROXY_URL}")
print("")
print("📝 Información:")
print("   - La API key de OpenAI está en el servidor proxy (oculta)")
print("   - Los estudiantes NO pueden ver ni copiar la API key")
print("   - El uso está controlado a través del proxy")
print("   - Servidor desplegado en Render.com (gratis)")
print("")
print("👉 Continúa ejecutando las siguientes celdas")
print("="*70)

## 🌐 Paso 6: Instalar y Configurar ngrok

ngrok nos permite exponer el servidor local a Internet de forma segura.

In [ ]:
%%bash
echo "🌐 Instalando ngrok..."

# Descargar ngrok
wget -q https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
tar xzf ngrok-v3-stable-linux-amd64.tgz
chmod +x ngrok
mv ngrok /usr/local/bin/

echo "✅ ngrok instalado"
ngrok version

## 🚀 Paso 7: Iniciar Servidor

**¡Última celda!** Esto iniciará el servidor y creará una URL pública.

⚠️ **IMPORTANTE:**
- Esta celda quedará ejecutándose (verás [*] a la izquierda)
- **NO la detengas** mientras uses la aplicación
- Busca la URL de ngrok en la salida (https://xxxx-xx-xxx.ngrok-free.app)
- Copia esa URL y ábrela en una nueva pestaña

In [ ]:
import subprocess
import time
import requests
from IPython.display import display, HTML
import os

print("🚀 Iniciando SimuPaciente UMH...")
print("="*70)

# Cambiar al directorio del simulador
os.chdir('/content/ECOE/simulador')

# PASO 1: Limpiar procesos anteriores
print("\n🧹 Limpiando procesos anteriores...")
subprocess.run(['pkill', '-f', 'colab_server.py'], stderr=subprocess.DEVNULL)
subprocess.run(['pkill', '-f', 'ngrok'], stderr=subprocess.DEVNULL)
time.sleep(2)
print("✅ Procesos limpiados")

# PASO 2: Iniciar ngrok en background
print("\n🌐 Iniciando túnel ngrok...")
with open('/tmp/ngrok.log', 'w') as ngrok_log:
    ngrok_process = subprocess.Popen(
        ['ngrok', 'http', '8080', '--log=stdout'],
        stdout=ngrok_log,
        stderr=subprocess.STDOUT
    )

print("⏳ Esperando a que ngrok establezca el túnel...")
time.sleep(8)

# PASO 3: Iniciar servidor Flask en background
print("\n🏥 Iniciando servidor Flask...")
with open('/tmp/flask.log', 'w') as flask_log:
    server_process = subprocess.Popen(
        ['python3', 'colab_server.py'],
        stdout=flask_log,
        stderr=subprocess.STDOUT
    )

print("⏳ Esperando a que el servidor esté listo...")
time.sleep(10)

# PASO 4: Verificar que el servidor está corriendo
server_ok = False
for attempt in range(5):
    try:
        response = requests.get('http://localhost:8080/api/cases', timeout=5)
        print("✅ Servidor Flask iniciado correctamente")
        server_ok = True
        break
    except:
        if attempt < 4:
            print(f"⏳ Intento {attempt + 1}/5...")
            time.sleep(3)

# PASO 5: Obtener URL de ngrok
public_url = None
for attempt in range(5):
    try:
        response = requests.get('http://localhost:4040/api/tunnels', timeout=3)
        tunnels = response.json()['tunnels']
        if tunnels:
            public_url = tunnels[0]['public_url']
            break
    except:
        if attempt < 4:
            time.sleep(2)

if public_url:
    print("\n" + "="*70)
    print("✅ ✅ ✅  SERVIDOR LISTO  ✅ ✅ ✅")
    print("="*70)
    print(f"\n🌍 URL Pública: {public_url}")
    print("\n📋 Instrucciones:")
    print("   1. Haz click en el botón de abajo")
    print("   2. ¡Comienza a usar SimuPaciente!")
    print("\n⚠️  El servidor está corriendo en background")
    print("⚠️  Para detenerlo: Runtime → Manage Sessions → Terminate")
    print("="*70)
    
    display(HTML(f'''
        <div style="
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            padding: 30px;
            border-radius: 15px;
            margin: 20px 0;
            text-align: center;
            box-shadow: 0 10px 30px rgba(0,0,0,0.3);
        ">
            <h2 style="color: white; margin-bottom: 20px; font-size: 24px;">🏥 SimuPaciente UMH</h2>
            <a href="{public_url}" target="_blank" style="
                display: inline-block;
                background: white;
                color: #667eea;
                padding: 15px 40px;
                text-decoration: none;
                border-radius: 25px;
                font-weight: bold;
                font-size: 18px;
                box-shadow: 0 5px 15px rgba(0,0,0,0.2);
            ">
                🚀 Abrir Aplicación
            </a>
            <p style="color: white; margin-top: 20px; font-size: 14px;">
                {public_url}
            </p>
        </div>
    '''))
    
    print("\n✅ Celda ejecutada exitosamente")
    print("📊 Logs del servidor en: /tmp/flask.log")
    print("🌐 Logs de ngrok en: /tmp/ngrok.log")
    
else:
    print("\n❌ No se pudo obtener la URL de ngrok")
    print("\n🔍 Revisa los logs:")
    print("   - Flask: /tmp/flask.log")
    print("   - ngrok: /tmp/ngrok.log")
    print("\n💡 Intenta ejecutar esta celda nuevamente")

---

## 📝 Notas Adicionales

### 🔄 Reiniciar el Servidor
Si necesitas reiniciar:
1. Detén la celda anterior (botón Stop)
2. Ejecuta la celda nuevamente

### 🐛 Solución de Problemas

**Problema:** No se conecta a OpenAI Realtime API
- Verifica que tu API key tiene acceso a Realtime API (beta)
- Comprueba que tienes créditos disponibles

**Problema:** La URL de ngrok no funciona
- Espera 30 segundos y recarga la página
- Verifica que la celda del servidor sigue corriendo [*]

### 📧 Soporte
- GitHub: https://github.com/marcosbenghezala/ECOE
- Universidad Miguel Hernández de Elche

---

**Versión:** 0.9.5 (Release Candidate)  
**Última actualización:** 17 de diciembre de 2025